# PORTFOLIO_REBALANCER

## Overview
The `PORTFOLIO_REBALANCER` function computes a new allocation of assets across Taxable, Roth, and Traditional accounts to move each asset class closer to its target allocation, while respecting account capacity constraints and prioritizing tax-inefficient assets for tax-advantaged accounts.

The function works as follows:
- For each asset class, it calculates the difference (delta) between the target allocation and the sum of the current allocations across all accounts.
- Asset classes are sorted by their tax inefficiency (highest tax score first), so that the most tax-inefficient assets are considered first for placement in tax-advantaged accounts.
- For each asset with a positive delta (needs more allocation), the function tries to add the required amount to Roth, then Traditional, then Taxable accounts, in that order, without exceeding the available capacity in each account.
- For each asset with a negative delta (needs to be reduced), the function removes the excess from Taxable, then Traditional, then Roth accounts, in that order, without reducing any account below zero.
- The function ensures that no allocation becomes negative and that account capacities are not exceeded.
- If an error occurs (e.g., invalid input), a string error message is returned.

This approach does not perform a full optimization or minimize taxes globally, but rather applies a greedy, rule-based reallocation that respects the order of tax efficiency and account limits. The function is intended for educational and illustrative purposes and may not produce the most tax-efficient result in all scenarios.

## Usage
To use the function in Excel:

```excel
=PORTFOLIO_REBALANCER(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap)
```

- `asset_classes` (list[list[str]], required): Asset class names as a list of lists, one per asset class. Example: [["US Stocks"],["Bonds"],["Cash"]]
- `current_alloc` (list[list[float]], required): Current dollar allocations for each asset in each account. Example: [[20000,5000,5000],[10000,2000,8000],[5000,1000,4000]]
- `target_alloc` (list[list[float]], required): Target total dollar allocation for each asset class. Example: [[30000],[20000],[10000]]
- `tax_scores` (list[list[float]], required): Tax efficiency score for each asset. Example: [[90],[60],[30]]
- `taxable_cap` (float, required): Maximum dollars available in the taxable account. Example: 30000
- `roth_cap` (float, required): Maximum dollars available in the Roth account. Example: 8000
- `trad_cap` (float, required): Maximum dollars available in the Traditional account. Example: 17000

The function returns a new allocation for each asset as a 2D list: [[taxable, roth, trad], ...]. If an error occurs, a string error message is returned.

## Examples

### Rebalancing a Three-Asset Portfolio

**Inputs:**

| Asset Class  | Taxable | Roth  | Trad  | Target | Tax Score |
|--------------|---------|-------|-------|--------|-----------|
| US Stocks    | 18000   | 7000  | 5000  | 25000  | 90        |
| Bonds        | 9000    | 3000  | 8000  | 20000  | 60        |
| Cash         | 4000    | 2000  | 4000  | 8000   | 30        |

- Taxable Cap: 30000
- Roth Cap: 8000
- Trad Cap: 17000

**Output:**

| Asset Class  | Taxable | Roth  | Trad  |
|--------------|---------|-------|-------|
| US Stocks    | 13000 | 7000 | 5000 |
| Bonds        | 9000  | 3000 | 8000 |
| Cash         | 2000  | 2000 | 4000 |


### Allocating All Assets to Taxable Accounts

**Inputs:**

| Asset Class | Taxable | Roth | Trad | Target | Tax Score |
|-------------|---------|------|------|--------|-----------|
| ETF1        | 8000    | 0    | 0    | 9000   | 10        |
| ETF2        | 2000    | 0    | 0    | 3000   | 10        |

- Taxable Cap: 12000
- Roth Cap: 1000
- Trad Cap: 1000

**Output:**

| Asset Class | Taxable | Roth | Trad |
|-------------|---------|------|------|
| ETF1        | 8000  | 1000 | 0 |
| ETF2        | 2000  | 0    | 1000 |

In [ ]:
# Function Implementation

def portfolio_rebalancer(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap):
    """
    Tax-efficient rebalancing of assets across Taxable, Roth, and Traditional accounts.
    Args:
        asset_classes (List[List[str]]): [[asset_class1], [asset_class2], ...]
        current_alloc (List[List[float]]): [[taxable, roth, trad], ...]
        target_alloc (List[List[float]]): [[target1], [target2], ...]
        tax_scores (List[List[float]]): [[score1], [score2], ...]
        taxable_cap (float): Max dollars in taxable account.
        roth_cap (float): Max dollars in Roth account.
        trad_cap (float): Max dollars in Traditional account.
    Returns:
        List[List[float]]: New allocation [[taxable, roth, trad], ...] or a string error message if an exception occurs.
    """
    try:
        # Assume all inputs are already 2D lists or floats
        n = len(asset_classes)
        asset_classes = [str(row[0]).strip() for row in asset_classes]
        current_alloc = [list(map(float, row)) for row in current_alloc]
        target_alloc = [float(row[0]) for row in target_alloc]
        tax_scores = [float(row[0]) for row in tax_scores]
        # Calculate deltas
        deltas = [target_alloc[i] - sum(current_alloc[i]) for i in range(n)]
        # Sort assets by tax inefficiency (descending)
        sorted_idx = sorted(range(n), key=lambda i: -tax_scores[i])
        # Start with current allocation
        new_alloc = [row[:] for row in current_alloc]
        # Account capacities
        taxable_left = taxable_cap
        roth_left = roth_cap
        trad_left = trad_cap
        for i in range(n):
            taxable_left -= current_alloc[i][0]
            roth_left -= current_alloc[i][1]
            trad_left -= current_alloc[i][2]
        # Allocate deltas
        for idx in sorted_idx:
            delta = deltas[idx]
            if abs(delta) < 1e-6:
                continue
            # Tax-inefficient assets: fill tax-advantaged first
            if delta > 0:
                # Add to Roth, then Trad, then Taxable
                add_roth = min(delta, roth_left)
                new_alloc[idx][1] += add_roth
                roth_left -= add_roth
                delta -= add_roth
                add_trad = min(delta, trad_left)
                new_alloc[idx][2] += add_trad
                trad_left -= add_trad
                delta -= add_trad
                add_taxable = min(delta, taxable_left)
                new_alloc[idx][0] += add_taxable
                taxable_left -= add_taxable
            else:
                # Remove from Taxable, then Trad, then Roth
                remove_taxable = min(-delta, new_alloc[idx][0])
                new_alloc[idx][0] -= remove_taxable
                delta += remove_taxable
                remove_trad = min(-delta, new_alloc[idx][2])
                new_alloc[idx][2] -= remove_trad
                delta += remove_trad
                remove_roth = min(-delta, new_alloc[idx][1])
                new_alloc[idx][1] -= remove_roth
                delta += remove_roth
        # Ensure no negative allocations
        for i in range(n):
            for j in range(3):
                if new_alloc[i][j] < 0:
                    new_alloc[i][j] = 0.0
        return [row[:] for row in new_alloc]
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

demo_cases = [
    # Example 1: Rebalancing a Three-Asset Portfolio (updated to be more realistic)
    [
        [["US Stocks"], ["Bonds"], ["Cash"]],
        [[18000, 7000, 5000], [9000, 3000, 8000], [4000, 2000, 4000]],  # Current allocation
        [[25000], [20000], [8000]],  # Target allocation (different from current)
        [[90], [60], [30]],
        30000,
        8000,
        17000,
    ],
    # Example 2: Allocating All Assets to Taxable Accounts
    [
        [["ETF1"], ["ETF2"]],
        [[8000, 0, 0], [2000, 0, 0]],
        [[9000], [3000]],
        [[10], [10]],
        12000,
        1000,
        1000,
    ],
]


def is_valid_type(val):
    if isinstance(val, (float, bool, str)):
        return True
    if isinstance(val, list):
        return all(
            isinstance(row, list) and all(isinstance(x, (float, bool, str)) for x in row)
            for row in val
        )
    return False


import pytest

@pytest.mark.parametrize(
    "asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap",
    demo_cases,
)
def test_demo_cases(
    asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap
):
    result = portfolio_rebalancer(
        asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap
    )
    print("Input:")
    print("asset_classes:", asset_classes)
    print("current_alloc:", current_alloc)
    print("target_alloc:", target_alloc)
    print("tax_scores:", tax_scores)
    print("taxable_cap:", taxable_cap)
    print("roth_cap:", roth_cap)
    print("trad_cap:", trad_cap)
    print("Output:")
    print(result)
    assert is_valid_type(result), f"Output type is not valid. Got: {type(result)} Value: {result}"

ipytest.run("-s")

In [ ]:
# Interactive Demo
import gradio as gr

demo = gr.Interface(
    fn=portfolio_rebalancer,
    inputs=[
        gr.Dataframe(headers=["Asset Classes"], label="Asset Classes", row_count=1, col_count=1, type="array", value=demo_cases[0][0]),
        gr.Dataframe(headers=["Taxable", "Roth", "Trad"], label="Current Allocation", row_count=1, col_count=3, type="array", value=demo_cases[0][1]),
        gr.Dataframe(headers=["Target"], label="Target Allocation", row_count=1, col_count=1, type="array", value=demo_cases[0][2]),
        gr.Dataframe(headers=["Tax Score"], label="Tax Scores", row_count=1, col_count=1, type="array", value=demo_cases[0][3]),
        gr.Number(label="Taxable Cap", value=demo_cases[0][4]),
        gr.Number(label="Roth Cap", value=demo_cases[0][5]),
        gr.Number(label="Trad Cap", value=demo_cases[0][6]),
    ],
    outputs=gr.Dataframe(headers=["Taxable", "Roth", "Trad"], label="New Allocation"),
    examples=demo_cases,
    description="Calculate a tax-efficient rebalancing of assets across Taxable, Roth, and Traditional accounts.",
    flagging_mode="never",
    fill_width=True,
)
demo.launch()